<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Number_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# 2. Baixando a CNN pré-treinada

In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

# 3. Conectando o drive ao colab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 4. Carregando os arquivos do colab e movendo para pasta number


In [0]:
!unzip "/content/drive/My Drive/Colab/number.zip"

# 5. Treinando o modelo

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="number")
trainer.setTrainConfig(object_names_array=["0","1", "2", "3", "4", "5", "6", "7", "8", "9"], batch_size=4, num_experiments=60, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

# 6. Avaliando o modelo gerado

In [0]:
metrics = trainer.evaluateModel(model_path="number/models", json_path="number/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

# 7. Contar numero de arquivos na pasta Imagens

In [0]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/images"))
file_count = len(files)
print(file_count)

# 8. Testando o modelo treinado

In [0]:
from imageai.Detection.Custom import CustomObjectDetection
import json

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("/content/drive/My Drive/Colab/models/detection_model-76.h5")
detector.setJsonPath("/content/drive/My Drive/Colab/json/detection_config.json")
detector.loadModel()

for x in range(file_count):

  if x in {0,1,2,3,4,5,6,7,8,9}:
    x = '0'+str(x)
  
  detections = detector.detectObjectsFromImage(input_image="/content/drive/My Drive/Colab/images/barcode-000" +str(x)+ ".jpg", 
                                          output_image_path="/content/drive/My Drive/Colab/images_detected/digit-detected-" +str(x)+ ".jpg", 
                                          display_percentage_probability=False, 
                                          minimum_percentage_probability=90)
  def boxPoints(e):
    return e['box_points']

  detections.sort(key=boxPoints)

  print("")
  print("Número " , ": ", "Porcentagem ", "      :    ", "box_points")
  print("")

  i=0
  soma=0

  for detection in detections:
        print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])
        soma = soma + detections[i]["percentage_probability"]
        i=i+1

  media = soma/len(detections)
  print("")
  print("media: ", media)
  print("")
  print("Quantidade de números reconhecidos na tag: ", len(detections))
  print("")
  print("----------------------------")

  media = {"media": media}
  detections.append(media)

  with open('/content/drive/My Drive/Colab/Corridas/Corrida_1/tag-'+ str(x) +'.json', 'w', encoding='utf-8') as f:
      json.dump(detections, f, ensure_ascii=False, indent=4)

In [0]:
detections[2]["name"] == detections[3]["name"]

# 8. Baixe o modelo para utilizar em outras ocasiões e reconhecendo os números em uma nova imagem

In [0]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("drive/My Drive/Colab/models/-----NOME DO MODELO-----")
detector.setJsonPath("drive/My Drive/Colab/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image="drive/My Drive/Colab/images/barcode-00015.jpg", output_image_path="drive/My Drive/Colab/images_detected/digit-detected-15.jpg")
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])